Single Server

In [23]:
import numpy as np
import pandas as pd

def simulate_service(num_customers, mean_arrival_time, mean_service_time):
    # Generate arrival times and service times from exponential distributions
    arrival_times = np.cumsum(np.random.exponential(mean_arrival_time, num_customers))
    service_times = np.random.exponential(mean_service_time, num_customers)

    # Initialize arrays to store exit times, queueing times, and server idle times
    exit_times = np.zeros(num_customers)
    queueing_times = np.zeros(num_customers)
    server_idle_times = np.zeros(num_customers)

    # Simulate the service mechanism
    for i in range(num_customers):
        # Calculate queueing time (if any)
        if i > 0 and exit_times[i - 1] > arrival_times[i]:
            queueing_times[i] = exit_times[i - 1] - arrival_times[i]

        # Calculate exit time
        exit_times[i] = arrival_times[i] + service_times[i] + queueing_times[i]

        # Calculate server idle time (if any)
        if i > 0 and exit_times[i - 1] < arrival_times[i]:
            server_idle_times[i] = arrival_times[i] - exit_times[i - 1]

    total_time_in_system = service_times + queueing_times

    return pd.DataFrame({
        'Customer': range(1, num_customers + 1),
        'Arrival Time': arrival_times,
        'Service Time': service_times,
        'Exit Time': exit_times,
        'Queueing Time': queueing_times,
        'Server Idle Time': server_idle_times,
        'Total Time in System': total_time_in_system
    })

# Example usage
num_customers = 10
mean_arrival_time = 5.0  # Mean arrival time for exponential distribution
mean_service_time = 3.0  # Mean service time for exponential distribution

simulation_result = simulate_service(num_customers, mean_arrival_time, mean_service_time)

print(simulation_result.to_string(index=False))


 Customer  Arrival Time  Service Time  Exit Time  Queueing Time  Server Idle Time  Total Time in System
        1      2.720189      1.928354   4.648543       0.000000          0.000000              1.928354
        2      4.143496      0.308871   4.957414       0.505047          0.000000              0.813918
        3      6.343566      2.029554   8.373120       0.000000          1.386153              2.029554
        4     13.434476      1.924351  15.358827       0.000000          5.061355              1.924351
        5     13.506966      0.570584  15.929411       1.851861          0.000000              2.422445
        6     14.123868      1.706697  17.636108       1.805543          0.000000              3.512240
        7     14.359340      1.525010  19.161118       3.276768          0.000000              4.801778
        8     14.567247      2.870167  22.031285       4.593871          0.000000              7.464038
        9     24.238262      3.024344  27.262606       0.000000 

Multiple Server

In [24]:
import numpy as np
import pandas as pd

def simulate_service(num_customers, num_servers, mean_arrival_time, mean_service_time):
    # Generate arrival times and service times from exponential distributions
    arrival_times = np.cumsum(np.random.exponential(mean_arrival_time, num_customers))
    service_times = np.random.exponential(mean_service_time, num_customers)

    # Initialize arrays to store exit times, queueing times, and server idle times
    exit_times = np.zeros(num_customers)
    queueing_times = np.zeros(num_customers)
    server_idle_times = np.zeros(num_customers)

    # Initialize server availability status
    server_available = np.ones(num_servers, dtype=bool)

    # Simulate the service mechanism
    for i in range(num_customers):
        # Find an available server or join the queue
        server_index = np.where(server_available)[0]
        if len(server_index) > 0:
            # Assign the customer to an available server
            server_id = server_index[0]
            server_available[server_id] = False

            # Calculate queueing time (if any)
            if i > 0 and exit_times[i - 1] > arrival_times[i]:
                queueing_times[i] = exit_times[i - 1] - arrival_times[i]

            # Calculate exit time
            exit_times[i] = max(arrival_times[i], exit_times[i - 1]) + service_times[i] + queueing_times[i]

            # Release the server after service completion
            server_available[server_id] = True
        else:
            # All servers are busy, customer joins the queue
            queueing_times[i] = exit_times[i - 1] - arrival_times[i]

            # Calculate exit time considering queueing time
            exit_times[i] = arrival_times[i] + service_times[i] + queueing_times[i]

        # Calculate server idle time (if any)
        if i > 0 and exit_times[i - 1] < arrival_times[i]:
            server_idle_times[i] = arrival_times[i] - exit_times[i - 1]

    total_time_in_system = service_times + queueing_times

    return pd.DataFrame({
        'Customer': range(1, num_customers + 1),
        'Arrival Time': arrival_times,
        'Service Time': service_times,
        'Exit Time': exit_times,
        'Queueing Time': queueing_times,
        'Server Idle Time': server_idle_times,
        'Total Time in System': total_time_in_system
    })

# Example usage
num_customers = 10
num_servers = 2
mean_arrival_time = 5.0  # Mean arrival time for exponential distribution
mean_service_time = 3.0  # Mean service time for exponential distribution

simulation_result = simulate_service(num_customers, num_servers, mean_arrival_time, mean_service_time)

print(simulation_result.to_string(index=False))


 Customer  Arrival Time  Service Time  Exit Time  Queueing Time  Server Idle Time  Total Time in System
        1      2.346920      8.451766  10.798686       0.000000               0.0              8.451766
        2      7.262545      2.570347  16.905175       3.536141               0.0              6.106489
        3     10.759742      1.473902  24.524510       6.145433               0.0              7.619335
        4     20.466490      3.092481  31.675012       4.058021               0.0              7.150502
        5     25.841363      1.838868  39.347529       5.833649               0.0              7.672517
        6     26.730628      2.366443  54.330873      12.616901               0.0             14.983344
        7     27.096540      8.514381  90.079587      27.234333               0.0             35.748714
        8     32.238511      1.463783 149.384446      57.841076               0.0             59.304859
        9     32.372856      9.747276 276.143312     117.011590 